source .venv/bin/activate

In [9]:
from _core.paths import *
from _core.libs import *

In [10]:
!python -m _tools.update_universe

Запрашиваю актуальный список акций с Московской биржи...
✅ Получено 258 активных тикеров.
✅ Список тикеров успешно сохранен в файл: /home/restorator/trader_test/universe.csv


In [33]:
!python -m _tools.download_all_data

--- Начинаю полную загрузку данных за 20 лет ---

[1/3] Загрузка котировок по акциям...
Загрузка акций: 100%|█████████████████████████| 258/258 [00:17<00:00, 15.03it/s]

[2/3] Загрузка истории индекса IMOEX...
Загрузка истории индекса IMOEX с 2005-09-12...
✅ Полная история индекса IMOEX (5012 строк) сохранена в 1_data/IMOEX.csv

[3/3] Загрузка истории ключевой ставки ЦБ РФ...
Загрузка полной истории ключевой ставки со страницы: https://www.cbr.ru/hd_base/keyrate/...
/home/restorator/trader_test/_tools/download_cbr_rate.py:67: ChainedAssignmentError: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
When using the Copy-on-Write mode, such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to per

Очистить все бумаги, обрезая по 2 дня около длинных разрывов, с минимальным сегментом ≥5 дней:

In [ ]:
!python -m _tools.clean_timeseries --data-dir 1_data --gap-ok 3 --gap-buffer 2 --min-seg-len 5 --inplace

Cleaning OHLCV: 100%|█████████████████████████| 258/258 [00:05<00:00, 47.05it/s]

[OK] Report saved to /home/restorator/trader_test/2_results/clean_report.csv


In [11]:
# --- ⚙️ КОНФИГУРАЦИЯ ---
MODEL_NAME = "universal_v2.0_60_20"
UNIVERSE_FILE = "universe.csv"

MODEL_NAME_LGBM = f"{MODEL_NAME}_lgbm"
MODEL_NAME_PERM = f"{MODEL_NAME}_perm"
MODEL_NAME_HYBRID = f"{MODEL_NAME}_hybrid"

In [27]:
import importlib, _core.feature_generator as fg
importlib.reload(fg)

df = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/trader_test/1_data/GAZP_D1_MOEX.csv",
    parse_dates=["datetime"], index_col="datetime"
)

feats = fg.create_features(df)
print("Размер:", feats.shape)
print("Количество фич:", len(feats.columns))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/trader_test/1_data/GAZP_D1_MOEX.csv'

In [14]:
import importlib, inspect, _core.feature_generator as fg
print("MODULE FILE:", fg.__file__)                 # путь к реально импортированному файлу
print("HAS add_all_ta_features CALL:",
      "add_all_ta_features" in inspect.getsource(fg.create_features))  # True/False
importlib.reload(fg)  # жёсткий перезагруз


MODULE FILE: /home/restorator/trader_test/_core/feature_generator.py
HAS add_all_ta_features CALL: False


<module '_core.feature_generator' from '/home/restorator/trader_test/_core/feature_generator.py'>

In [13]:
import importlib
import _core.feature_generator as fg

importlib.reload(fg)

df = fg.normalize_columns(df)
print(df.columns)


NameError: name 'df' is not defined

In [12]:
!python -m _tools.feature_selector \
  --model_name "{MODEL_NAME_LGBM}" \
  --universe_file "{UNIVERSE_FILE}"

I0000 00:00:1757613033.773834   41430 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1757613035.008113   41430 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
✅ Окружение и сторонние библиотеки инициализированы.
Найдено 258 тикеров для обработки.
--- Подготовка данных по индексу IMOEX ---
--- Сбор данных для обучения селектора ---
Сбор данных:   0%|                                      | 0/258 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/home/restorator/trader_test/.venv-tf/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3812, in get_loc
    return self._engi

In [7]:
!python -m _tools.feature_selector_perm \
  --model_name "{MODEL_NAME_PERM}" \
  --universe_file "{UNIVERSE_FILE}" \
  --top_n 25 \
  --n_splits 5 \
  --purge_gap 20 \
  --repeats 10 \
  --corr_threshold 0.9 \
  --boots 1 \
  --bootstrap_frac 1.0

2025-09-10 10:06:28.353333: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757498788.372183    2814 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757498788.378001    2814 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757498788.399288    2814 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757498788.399320    2814 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757498788.399324    2814 computation_placer.cc:177] computation placer alr

In [8]:
!python -m _tools.feature_selector_hybrid \
  --model_name_a "{MODEL_NAME_LGBM}" \
  --model_name_b "{MODEL_NAME_PERM}" \
  --out_model_name "{MODEL_NAME_HYBRID}" \
  --universe_file "{UNIVERSE_FILE}" \
  --top_n 25

2025-09-10 10:40:30.170699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757500830.189733   11528 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757500830.195669   11528 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757500830.210594   11528 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757500830.210625   11528 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757500830.210629   11528 computation_placer.cc:177] computation placer alr

In [16]:
MODEL_BASE = "universal_v3.0_60_20"          # базовое имя версии
MODEL_NAME = f"{MODEL_BASE}_hybrid"          # работаем с гибридным списком фич
UNIVERSE_FILE = "universe.csv"               # корневой путь к файлу с тикерами

# быстрый чек наличия списка фич
import os, json
feat_path = os.path.join("2_results", f"selected_features_{MODEL_NAME}.json")
assert os.path.exists(feat_path), f"Не найден список фич: {feat_path}"
with open(feat_path) as f:
    feats = json.load(f)
print(f"✅ {len(feats)} фич • {feat_path}")

✅ 25 фич • 2_results/selected_features_universal_v3.0_60_20_hybrid.json


In [ ]:
!python -m _tools.convert_to_tfrecord \
  --input_model_name "{MODEL_NAME}" \
  --output_model_name "{MODEL_NAME}" \
  --universe_file "{UNIVERSE_FILE}"

In [17]:
!python -m _tools.train_universal_lstm --model_name "{MODEL_NAME}"

2025-09-10 11:04:37.488671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757502277.507834   17634 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757502277.513341   17634 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757502277.527771   17634 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757502277.527798   17634 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757502277.527802   17634 computation_placer.cc:177] computation placer alr

In [ ]:
!python -m _tools.train_universal_transformer --model_name "{MODEL_NAME}"

In [12]:
!python -m _tools.report_extrema_accuracy --model_name "{MODEL_NAME}" --universe_file "{UNIVERSE_FILE}"

2025-09-10 11:01:27.434175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757502087.452918   16796 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757502087.458716   16796 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757502087.473388   16796 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757502087.473414   16796 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757502087.473417   16796 computation_placer.cc:177] computation placer alr

In [ ]:
OLD_REPORT = "2_results/report_universal_v2_60_20_on_universe.csv"
NEW_REPORT = f"2_results/report_{MODEL_NAME}_on_universe.csv"

!python -m _tools.compare_reports "{OLD_REPORT}" "{NEW_REPORT}"

✅ Окружение и сторонние библиотеки инициализированы.
✅ Файлы 'report_universal_v2.1_60_20_on_universe.csv' и 'report_universal_v2_60_20_on_universe.csv' успешно загружены.

--- Сводная таблица сравнения моделей (медианные значения) ---
                     Точность (Precision) Полнота (Recall) F1-Score
universal_v2.1_60_20               76.78%            6.11%    11.18
universal_v2_60_20                 50.00%           25.42%    34.09

--- Детальный анализ по акциям (изменение F1-Score) ---
Улучшение F1-Score: 228 акций
Ухудшение F1-Score: 0 акций
Без изменений: 0 акций
✅ График сравнения сохранен в файл: 2_results/comparison_universal_v2.1_60_20_vs_universal_v2_60_20.png
